In [24]:
from prettytable import PrettyTable
from helpers import vault
from helpers import elastic 
from helpers import vault
import getpass 
import os

# Obtain the necessary values
VAULT_ADDR = os.getenv('VAULT_ADDR') or getpass.getpass("Enter Vault URL: ")
VAULT_SECRET_PATH = "/secrets/data/incident-response/jupyter-notebooks"
VAULT_TOKEN = open(os.path.expanduser('~/.vault-token')).read() or os.getenv('VAULT_TOKEN') or getpass.getpass("Enter Vault token: ")

# Get and extract Vault secrets
vault_secrets = vault.GetVaultSecrets(VAULT_ADDR, VAULT_SECRET_PATH, VAULT_TOKEN)
elasticsearch_username = vault_secrets['elasticsearch-username']
elasticsearch_password = vault_secrets['elasticsearch-password']
elasticsearch_hostname = vault_secrets['elasticsearch-hostname']

# Elasticsearch query to look for Powershell Empire
query_body = {
  "query": {
    "bool": {
      "must": [
            {
              "match": {
                "process.command_line": "powershell -noP -sta -w 1 -enc"
               }
            },
            {
              "match": {
                "process.name": "powershell.exe"
              }
            },
            {
              "match": {
                "event.code": "1"
              }
            }
        ]
    }
  }
}

# Init ES connector
es = elastic.InitESconnctor(elasticsearch_hostname, elasticsearch_username, elasticsearch_password, port=9200, scheme="http")

# Get Elasticsearch from user
elasticsearch_index = input("Enter Elasticsearch index to search: ")

# Get results
results = elastic.QueryElasticsearch(es, elasticsearch_index, query_body)

x = PrettyTable()
x.field_names = ['Sysmon code', 'Hostname',  'parent process name', 'ppid', 'process name', 'pid', 'Command line', 'Document ID']
x._max_width = {"Command line" : 40}
for entry in results['hits']['hits']:
    elastic_doc_id = entry['_id']
    
    event_code = entry['_source']['winlog']['event_id']
    hostname = entry['_source']['host']['hostname']
    
    parent_proc_name = entry['_source']['process']['parent']['name']
    ppid = entry['_source']['process']['parent']['pid']
    proc_name = entry['_source']['process']['name']
    pid = entry['_source']['process']['pid']
    cmd = entry['_source']['process']['command_line']
    x.add_row([event_code, hostname, parent_proc_name, ppid, proc_name, pid, cmd[:100], elastic_doc_id])
print(x)


https://vault.hackinglab.local/v1/secrets/data/incident-response/jupyter-notebooks
Enter Elasticsearch index to search: bg-sysmon
+-------------+----------+---------------------+------+----------------+------+------------------------------------------+----------------------+
| Sysmon code | Hostname | parent process name | ppid |  process name  | pid  |               Command line               |     Document ID      |
+-------------+----------+---------------------+------+----------------+------+------------------------------------------+----------------------+
|      1      |  saturn  |     WmiPrvSE.exe    | 1652 | powershell.exe | 3956 | powershell -noP -sta -w 1 -enc  SQBGACgA | WBXAWHMB43T4WvXhrRxh |
|             |          |                     |      |                |      | JABQAFMAVgBFAHIAcwBJAE8ATgBUAGEAYgBMAEUA |                      |
|             |          |                     |      |                |      |           LgBQAFMAVgBFAHIAUwBp           |                  

In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))